# Introduction to pygeoda 0.0.3

Xun Li 17/10/2019

pygeoda is a R package that wraps all core functions of spatial data
analysis in GeoDa and libgeoda. Unlike the desktop software GeoDa,
libgeoda is a non-UI and feature focused C++ library that is designed
for programmers to do spatial data analysis using their favoriate
programming languages (R, Python, Java etc.). It also aims to be easily
integratd with other libraries, softwares or systems on different
platforms.

This book is used to introduce the pygeoda v0.0.3 library, includes all
the functions that are currently provided in version 0.0.3.

**Live Tutorials**

There are python jupyter notebooks available to replicate all the content of this book: https://github.com/lixun910/pygeoda_tutorial/tree/v0.0.3

You can try these python jupyter notebooks in your browser via MyBinder (no installation required): 
https://mybinder.org/v2/gh/lixun910/pygeoda/v0.0.3

## 1. Install pygeoda v0.0.3

Like GeoDa desktop software, `pygeoda` are avaiable to different platforms including: Mac, Linux and Windows. 

In current development stage, `pygeoda` 0.0.3 can be installed from source code or using `pip`.

> Note: check pip page `pygeoda 0.0.3` at: https://pypi.org/project/pygeoda/


### 1.1 Install pygeoda using pip

You can use `pip` to install pygeoda 0.0.3:
```Shell
pip install pygeoda
```

### 1.2 Install from source

If pip install doesn't work, you can also try to install pygeoda from its source code:
```Shell
    pip install git+https://github.com/lixun910/pygeoda
```


## 2. Using pygeoda

If everything installed without error, you should be able to load pygeoda:


In [1]:
import pygeoda

### 2.1 Load Spatial Data

The data formats that pygeoda v0.0.3 can load directly includes:

* ESRI Shapefile
* MapInfo File
* CSV
* GML
* GPX
* KML
* GeoJSON
* TopoJSON
* OpenFileGDB
* GFT Google Fusion Tables
* CouchDB

In this tutorial, we only tested loading ESRI shapefiles
    using pygeoda v0.0.3. Please create a ticket in pygeoda's
    repo https://github.com/lixun910/pygeoda/issues if you
    experience any issues when loading spatial data.
    
For example, to load the ESRI Shapefile **Guerry.shp**
download from: https://geodacenter.github.io/data-and-lab/Guerry  


In [2]:
guerry = pygeoda.open("./data/Guerry.shp")

The `pygeoda.open()` function returns a geoda object, which
can be used to access the meta-data, fields, and columns of the input dataset.
```
    Usage
    pygeoda.open(ds_path)

    Arguments
    ds_path	(str) The path of the spatial dataset

    Value
    gda_obj An object of geoda class
```

#### Attributes of `geoda` object

* n_cols
* n_obs
* field_names
* field_type

To access the meta-data of the loaded Guerry dataset:

In [3]:
print("number of columns:", guerry.num_cols)
print("number of observations:", guerry.num_obs)
print("field names:", guerry.field_names)
print("field types:", guerry.field_types)

number of columns: 26
number of observations: 85
field names: ('CODE_DE', 'COUNT', 'AVE_ID_', 'dept', 'Region', 'Dprtmnt', 'Crm_prs', 'Crm_prp', 'Litercy', 'Donatns', 'Infants', 'Suicids', 'MainCty', 'Wealth', 'Commerc', 'Clergy', 'Crm_prn', 'Infntcd', 'Dntn_cl', 'Lottery', 'Desertn', 'Instrct', 'Prsttts', 'Distanc', 'Area', 'Pop1831')
field types: ('string', 'numeric', 'numeric', 'integer', 'string', 'string', 'integer', 'integer', 'integer', 'integer', 'integer', 'integer', 'integer', 'integer', 'integer', 'integer', 'integer', 'integer', 'integer', 'integer', 'integer', 'integer', 'integer', 'numeric', 'integer', 'numeric')


### 2.2 Access Table Data

The `geoda` instance also provide functions to get data from the dataset:

* GetIntegerCol(col_name)
* GetRealCol(col_name)
* GetStringCol(col_name)

For example, to get the integer values of the "Crm_prp" column:



In [4]:
crm_prp = guerry.GetIntegerCol("Crm_prp")
print("firs 10 values of Crm_prp:", crm_prp[:10])

firs 10 values of Crm_prp: (15890, 5521, 7925, 7289, 8174, 10263, 8847, 9597, 4086, 10431)


## 2.3 Spatial Weights

Spatial weights are central components in spatial data analysis. The spatial weights represents the possible spatial interaction between observations in space. Like GeoDa desktop software, `pygeoda` provides a rich variety of methods to create several different types of spatial weights:

* Contiguity Based Weights: `queen_weights()`, `rook_weights()`
* Distance Based Weights: `distance_weights()`
* K-Nearest Neighbor Weights: `knn_weights()`
* Kernel Weights: `kernel_weights()`

### 2.3.1 Queen Contiguity Weights

To create a Queen contiguity weights, we can call pygeoda's function 
```python
pygeoda.weights.queen(gda, order=1,
              include_lower_order = False, 
              precision_threshold = 0)
``` 
by passing the GeoDa object `guerry` we just created: 

In [5]:
queen_w = pygeoda.weights.queen(guerry)
print(queen_w)

Weights Meta-data:

is symmetric:True
sparsity:0.0
density:5.813148788927336
min neighbors:2
mean neighbors:4.9411764705882355
median neighbors:5.0
max neighbors:8


The function `queen_weights()` returns an instance of 
`Weight` object. One can access the meta data of the spatial
weights by accessing the attributes of `Weight` object:


#### Attributes of `Weight` object

* num_obs
* GetWeightsType() 
* IsSymmetric()
* HasIsolates()
* GetSparsity()
* GetMinNbrs()
* GetMedianNbrs()
* GetMeanNbrs()
* GetMaxNbrs()
* GetDensity()
* [] GetNeighbors(idx)
* double SpatialLag(idx, [data])
* SaveToFile()

We can also access the details of the weights: e.g.
list the neighbors of a specified observation, which
is very helpful in exploratory spatial data analysis:

In [6]:
nbrs = queen_w.GetNeighbors(0)
print("Neighbors of 0-st observation are:", nbrs)

Neighbors of 0-st observation are: (35, 36, 66, 68)


We can also compute the spatial lag of a specified observation by passing the values of the selected variable:

In [7]:
lag0 = queen_w.SpatialLag(0, crm_prp)
print("Spatial lag of 0-st observation is:", lag0)

Spatial lag of 0-st observation is: 7899.25


by passing the geoda object `guerry` we just created:
### 2.3.2 Rook Contiguity Weights

To create a Rook contiguity weights, we can call pygeoda's function 
```python
pygeoda.weights.rook(gda, order=1, 
                     include_lower_order=False, 
                     precision_threshold = 0)
``` 
by passing the GeoDa object `guerry` we just created: 

In [8]:
rook_w = pygeoda.weights.rook(guerry)
print(rook_w)

Weights Meta-data:

is symmetric:True
sparsity:0.0
density:5.813148788927336
min neighbors:2
mean neighbors:4.9411764705882355
median neighbors:5.0
max neighbors:8


The weights we created are in memory, which makes it straight forward for spatial data analysis and also are good for programming your application. To save the weights to a file, we need to call GeoDaWeight's function 
```python
Weight.SaveToFile(ofname, layer_name, id_name, id_vec)
```

The `layer_name` is the layer name of loaded dataset. For a ESRI shapefile, the layer name is the file name without the suffix (e.g. Guerry). 

The `id_name` is a key (column name), which means the associated column contains unique values, that makes sure that the weights are connected to the correct observations in the data table. 

The `id_vec` is the actual column data of `id_name`, it could be a tuple of integer or string values.

For example, in Guerry dataset, the column "CODE_DE" can be used as a key to save a weights file:

In [9]:
rook_w.SaveToFile('./data/Guerry_r.gal', 'Guerry', 'CODE_DE', guerry.GetIntegerCol('CODE_DE'))

True

Then, we should find the file "Guerry_r.gal" in the same directory of Guerry shapefiles.

### 2.3.3 Distance Based Weights

To create a Distance based weights, we can call pygeoda's function 
```python
pygeoda.weights.distance(gda, dist_thres, power=1.0, 
                         is_inverse=False, is_arc=False, 
                         is_mile=True)
``` 
by passing the GeoDa object `guerry` we just created and the value of distance threshold. Like GeoDa, pygeoda provides a function to help you find a optimized distance threshold that guarantees that every observation has at least one neighbor:

```python
pygeoda.weights.min_threshold(GeoDa gda, bool is_arc = False, bool is_mile = True)
```

In [10]:
dist_thres = pygeoda.weights.min_threshold(guerry)
dist_w = pygeoda.weights.distance(guerry, dist_thres)
print(dist_w)

Weights Meta-data:

is symmetric:False
sparsity:0.0
density:4.346020761245675
min neighbors:1
mean neighbors:3.6941176470588237
median neighbors:4.0
max neighbors:7


### 2.3.4 K-Nearest Neighbor Weights

A special case of distance based weights is K-Nearest neighbor weights, in which every obersvation will have exactly k neighbors. To create a KNN weights, we can call pygeoda's function:
```python
pygeoda.weights.knn(gda, k, power = 1.0, 
                    is_inverse = False,
                    is_arc = False, is_mile = True)
```

For example, to create a 6-nearest neighbor weights using Guerry dataset:

In [11]:
knn6_w = pygeoda.weights.knn(guerry, 6)
print(knn6_w)

Weights Meta-data:

is symmetric:False
sparsity:0.0
density:7.0588235294117645
min neighbors:6
mean neighbors:6.0
median neighbors:6.0
max neighbors:6


### 2.3.5 Kernel Weights

Kernel weights apply kernel function to determine the distance decay in the derived continuous weights kernel. The kernel weights are defined as a function K(z) of the ratio between the distance dij from i to j, and the bandwidth hi, with z=dij/hi. 

The kernl functions include:
* triangular
* uniform 
* quadratic
* epanechnikov
* quartic
* gaussian

Two functions are provided in pygeoda to create kernel weights:

**Kernel Weights with fixed bandwidth**

To create a kernel weights with fixed bandwith, using the following function
```python
pygeoda.weights.kernel_bandwidth(gda, bandwidth, kernel,
                                 use_kernel_diagnals = False,
                                 is_arc = False, is_mile = True)
```



In [12]:
kernel_w = pygeoda.weights.kernel_bandwidth(guerry, dist_thres, "uniform")
print(kernel_w)

Weights Meta-data:

is symmetric:False
sparsity:0.0
density:4.346020761245675
min neighbors:1
mean neighbors:3.6941176470588237
median neighbors:4.0
max neighbors:7


Besides the options `is_inverse`, `power`, `is_arc` and
`is_mile` that are the same with the distance based weights,
this kernel weights function has another option
```
    use_kernel_diagonals
    (optional) FALSE (default) or TRUE, apply kernel on the
    diagonal of weights matri
```

**Kernel Weights with adaptive bandwidth**

To create a kernel weights with adaptive bandwidth or using max Knn distance as bandwidth, using the following function:
```python
pygeoda.weights.kernel(gda, k, kernel,
                       adaptive_bandwidth = True,
                       use_kernel_diagnals = False,
                       is_arc = False, is_mile = True)
```

In [13]:
adptkernel_w = pygeoda.weights.kernel(guerry, 6, "uniform")
print(adptkernel_w)

Weights Meta-data:

is symmetric:False
sparsity:0.0
density:7.0588235294117645
min neighbors:6
mean neighbors:6.0
median neighbors:6.0
max neighbors:6


This kernel weights function two more options:
```

    adaptive_bandwidth
    (optional) TRUE (default) or FALSE: TRUE use adaptive bandwidth
    calculated using distance of k-nearest neithbors, FALSE use max
    distance of all observation to their k-nearest neighbors

    use_kernel_diagonals
    (optional) FALSE (default) or TRUE, apply kernel on the diagonal
    of weights matrix
```

## 3 Spatial Data Analysis


### 3.1 Local Spatial Autocorrelation

pygeoda 0.0.3 provids following methods for univariate
local spatial autocorrelation statistics:

* Local Moran: local_moran()
* Local Geary: local_geary()
* Local Getis-Ord statistics: local_g() and local_gstar()
* Local Join Count: local_joincount()


Methods for bivariate and multivariate local spatial autocorrelation statistics, as well as global spatial autocorrelation satatistics, will be included in next release of pygeoda.

In this tutorial, we will only introduce how to call these methods using pygeoda. For more information about the local spatial autocorrelation statisticis, please read: http://geodacenter.github.io/workbook/6a_local_auto/lab6a.html. 

#### 3.1.1 Local Moran

The Local Moran statistic is a method to identify local clusters and local spatial outliers. For example, we can call  function `local_moran()` with the created Queen weights and the data “crm_prp” as input parameters:


In [14]:
lisa = pygeoda.local_moran(queen_w, crm_prp)

The `local_moran()` function will return a `lisa` object,
which we can call its functions to access the results of lisa
computation. The functions include:

* GetClusterIndicators(): Get the local cluster indicators returned from LISA computation.
* GetColors(): Get the cluster colors of LISA computation.
* GetLabels(): Get the cluster labels of LISA computation.
* GetLISAValues(): Get the local spatial autocorrelation values returned from LISA computation.
* GetNumNeighbors(): Get the number of neighbors of every observations in LISA computation.
* GetPValues(): Get the local pseudo-p values of significance returned from LISA computation.
* SetPermutations(num_perm): Set the number of permutations for the LISA computation
* SetThreads(num_threads): Set the number of CPU threads for the LISA computation
* Run(): Call to run LISA computation

For example, we can call the function `GetLISAValues()`
to get the values of local Moran:

In [15]:
lms = lisa.GetLISAValues()
print(lms[:20])

(0.015431978309803657, 0.3270633223656033, 0.021295296214118884, 0.004610544790030418, -0.0028342407096540465, 0.41493771583040345, -0.13794630908086175, 0.09986576922564794, 0.2823176310018141, 0.1218745112146858, -0.09512054168698209, 0.032611193818022896, 0.3878324535340113, 1.1888723840162665, -0.6452792226077357, -0.30964927402750314, 0.3662775143008573, 2.0375343538940496, -0.005015479968822494, 0.06971105719113387)


To get the pseudo-p values of significance of local Moran computation:

In [16]:
pvals = lisa.GetPValues()
print(pvals[:20])


(0.414, 0.123, 0.001, 0.474, 0.452, 0.087, 0.243, 0.326, 0.299, 0.303, 0.237, 0.46, 0.258, 0.018, 0.199, 0.188, 0.131, 0.004, 0.456, 0.342)


To get the cluster indicators of local Moran computation:

In [17]:
cats = lisa.GetClusterIndicators()
print(cats[:20])


(0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0)


The predefined values of the indicators of LISA cluster are:
```
0 Not significant
1 High-High
2 Low-Low
3 High-Low
4 Low-High
5 Neighborless
6 Undefined
```
which can be accessed via function `GetLabels()`:



In [18]:
lbls = lisa.GetLabels()
lbls

('Not significant',
 'High-High',
 'Low-Low',
 'High-Low',
 'Low-High',
 'Undefined',
 'Isolated')

Note: Different LISA objects (e.g. local_geary()) will return different
    labels and colors.

By default, the `local_moran()` function will run with some default parameters, e.g.:
```
permutation number: 999
seed for random number generator: 123456789
```
, which are identical to GeoDa desktop software so that we can replicate the results as using  GeoDa software. It is also easy to change the paremter and re-run the LISA computation by calling Run() function. 

For example, re-run the above local Moran example using 9999 permutations 

In [20]:
lisa.SetPermutations(9999)
lisa.Run()


Then, we can use the same `lisa` object to get the new results after 9999 permutations:


In [21]:
pvals = lisa.GetPValues()
print(pvals[:20])


(0.4187, 0.1265, 0.0004, 0.4679, 0.4545, 0.0728, 0.2312, 0.3071, 0.3115, 0.3088, 0.2187, 0.4803, 0.2623, 0.0113, 0.2, 0.1797, 0.1267, 0.0026, 0.4565, 0.3517)


pygeoda uses GeoDa’s C++ code, in which multi-threading is used to accelerate the computation of LISA. We can specify how many threads to run the computation:


In [23]:
lisa.SetThreads(4)
lisa.Run()

Get the new results after using different number of threads
 


In [25]:
pvals = lisa.GetPValues()
print(pvals[:20])

(0.4187, 0.1265, 0.0004, 0.4679, 0.4545, 0.0728, 0.2312, 0.3071, 0.3115, 0.3088, 0.2187, 0.4834, 0.2686, 0.0102, 0.2024, 0.1795, 0.1218, 0.0025, 0.459, 0.3588)


#### 3.1.2 Local Geary

Local Geary is a type of LISA that focuses on squared differences/dissimilarity. A small value of the local geary statistics suggest positive spatial autocorrelation, whereas large values suggest negative spatial autocorrelation. 

For example, we can call the function `local_geary()` with the created Queen weights and the data “crm_prp” as input parameters:

In [29]:
geary_crmprp = pygeoda.local_geary(queen_w, crm_prp)

To get the cluster indicators of the local Geary computation:


In [30]:
cats = geary_crmprp.GetClusterIndicators()
print(cats[:20])

(0, 2, 4, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 2)


To get the pseudo-p values of the local Geary computation:


In [33]:
pvals = geary_crmprp.GetPValues()
print(pvals[:20])

(0.398, 0.027, 0.025, 0.126, 0.017, 0.314, 0.61, 0.141, 0.284, 0.11, 0.559, 0.462, 0.211, 0.236, 0.249, 0.229, 0.069, 0.041, 0.205, 0.02)


#### 3.1.3 Local Getis-Ord Statistics

There are two types of local Getis-Ord statistics: one is computing a ratio of the weighted average of the values in the neighboring locations, not including the value at the location; while another type of statistic includes the value at the location in both numerator and denominator.

A value larger than the mean suggests a high-high cluster or hot spot, a value smaller than the mean indicates a low-low cluster or cold spot.

For example, we can call the function `local_g()` with the created Queen weights and the data “crm_prp” as input parameters:

In [34]:
localg_crmprp = pygeoda.local_g(queen_w, crm_prp)

To get the cluster indicators of the local G computation:


In [35]:
cats = localg_crmprp.GetClusterIndicators()
print(cats[:20])

(0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0)


To get the pseudo-p values of the local G computation:


In [36]:
pvals = localg_crmprp.GetPValues()
print(pvals[:20])

(0.414, 0.123, 0.001, 0.474, 0.452, 0.087, 0.243, 0.326, 0.299, 0.303, 0.237, 0.46, 0.258, 0.018, 0.199, 0.188, 0.131, 0.004, 0.456, 0.342)


For the second type of local Getis-Ord statistics, we can call the function `local_gstar()` with the created Queen weights and the data “crm_prp” as input parameters:

In [37]:
localgstar_crmprp = pygeoda.local_gstar(queen_w, crm_prp)

#### 3.1.4 Local Join Count


Local Join Count is a method to identify local clusters for binary data by using a local version of the so-called BB join count statistic. The statistic is only meaningful for those observations with value 1. 

For example, we can load the columbus dataset, and call the function `local_joincount()` with a Queen weights and the data “nsa”, which is a set of binary (0,1) values, as input parameters:

In [39]:
columbus = pygeoda.open('./data/columbus.shp')
columbus_w = pygeoda.weights.queen(columbus)
localjc_nsa = pygeoda.local_joincount(columbus_w, columbus.GetIntegerCol('nsa'))

To get the cluster indicators of the local Join Count computation:


In [40]:
cats = localjc_nsa.GetClusterIndicators()
print(cats[:20])

(0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0)


To get the pseudo-p values of the local Join Count computation:


In [41]:
pvals = localjc_nsa.GetPValues()
print(pvals[:20])

(0.213, 0.07, 0.017, 0.024, 0.001, 0.226, 0.055, 0.007, 0.006, 0.275, 0.017, 0.008, 0.043, 0.004, 0.002, 0.001, 0.147, 0.049, 0.087, 0.001)


To get the number of neighbors of the local Join Count computation:

In [44]:
nn = localjc_nsa.GetNumNeighbors()
print(nn[:20])

(2, 3, 4, 4, 8, 2, 4, 6, 8, 4, 5, 6, 4, 6, 6, 8, 3, 4, 3, 10)


### 3.2 Spatial Clustering

 

Spatial clustering aims to group of a large number of
geographic areas or points into a smaller number of regions
based on similiarities in one or more variables.
Spatially constrained clustering is needed when clusters are
required to be spatially contiguous.

In pygeoda v0.0.3, there are three different approaches
explicitly incorporate the contiguity constraint in the
optimization process: SKATER, Redcap and Max-p.
More more details, please read the lab note that
Dr. Luc Anselin wrote:
http://geodacenter.github.io/workbook/8_spatial_clusters/lab8.html 

For example, to apply spatial clustering on the Guerry dataset,
we use the queen weights to define the spatial contiguity
and select 6 variables for similarity measure:
"Crm_prs", "Crm_prp", "Litercy", "Donatns", "Infants", "Suicids".

The following code is used to get a 2D data list for
the selected variables:


In [46]:
select_vars = ["Crm_prs", "Crm_prp", "Litercy", "Donatns", "Infants", "Suicids"]
data = [guerry.GetRealCol(v) for v in select_vars]

#### 3.2.1 SKATER

The Spatial C(K)luster Analysis by Tree Edge Removal(SKATER)
algorithm introduced by Assuncao et al. (2006) is based on
the optimal pruning of a minimum spanning tree that reflects
the contiguity structure among the observations. It provides
an optimized algorithm to prune to tree into several clusters
that their values of selected variables are as similar as possible.

The SKATER function in pygeoda:
::

    pygeoda.skater(k, w, data, distance_method='euclidean', bound_vals = [],  min_bound = 0, random_seed=123456789)


Note: the parameters `distance_method`, `bound_vals`, `min_bound` and `random_seed` are optional.

Note: See [Max-p] for the usage of  `bound_vals` and `min_bound`.

For example, to create 4 spatially contiguous clusters using
`skater()` with Guerry dataset, the queen weights and the
values of the 6 selected variables

In [48]:
skater_clusters = pygeoda.skater(4, queen_w, data)
print(skater_clusters)

((15, 74, 16, 55, 60, 39, 68, 33, 17, 82, 81, 0, 2, 40, 20, 80), (46, 50, 34, 38, 69, 47, 58, 19, 32, 41, 53, 26), (23, 79, 3, 29, 61, 21, 44, 11, 28, 13, 30, 35, 76, 77, 43, 9, 27, 45, 31, 78, 4, 10, 66, 37, 5, 14, 7, 63, 62), (49, 52, 72, 84, 8, 57, 56, 59, 42, 1, 25, 51, 48, 54, 64, 75, 18, 83, 73, 36, 24, 71, 6, 67, 65, 70, 22, 12))


This `skater()` function returns a 2D tuple, which represents 4 clusters. Each cluster is composed by several contiguity areas, e.g. 15, 74, 16, 55, 60, 39, 68, 33, 17, 82, 81, 0, 2, 40, 20, 80

pygeoda also provides utility functions to compute some descriptive statistics of the clustering results, e.g. to compute the ratio of between to total sum of squares:

In [50]:
betweenss = pygeoda.between_sumofsquare(skater_clusters, data)
totalss = pygeoda.total_sumofsquare( data)
ratio =  betweenss / totalss
print("The ratio of between to total sum of square:", ratio)

The ratio of between to total sum of square: 0.3156446659311204


#### 3.2.2 REDCAP

REDCAP (Regionalization with dynamically constrained agglomerative clustering and partitioning) is developed by D. Guo (2008). Like SKATER, REDCAP starts from building a spanning tree with 3 different ways (single-linkage, average-linkage, and the complete-linkage). The single-linkage way leads to build a minimum spanning tree. Then,REDCAP provides 2 different ways (first‐order and full-order constraining) to prune the tree to find clusters. The first-order approach with a minimum spanning tree is exactly the same with SKATER. In GeoDa and pygeoda, the following methods are provided:

* First-order and Single-linkage
* Full-order and Complete-linkage
* Full-order and Average-linkage
* Full-order and Single-linkage

For example, to find same 4 clusters using the same dataset and weights as above using Full-order and Complete-linkage REDCAP:

In [51]:
redcap_clusters = pygeoda.redcap(4, queen_w, data, "fullorder-completelinkage")
print(redcap_clusters)

betweenss = pygeoda.between_sumofsquare(redcap_clusters, data)
totalss = pygeoda.total_sumofsquare( data)
ratio =  betweenss / totalss
print("The ratio of between to total sum of square:", ratio)

((13, 60, 76, 77, 43, 39, 10, 5, 2), (20,), (40,), (49, 52, 46, 50, 34, 38, 72, 84, 23, 79, 57, 56, 59, 29, 61, 1, 15, 74, 21, 44, 11, 3, 25, 42, 51, 8, 48, 54, 64, 16, 55, 18, 28, 35, 9, 27, 45, 69, 31, 47, 58, 36, 83, 19, 32, 24, 71, 6, 73, 41, 78, 75, 68, 33, 65, 70, 66, 53, 30, 37, 17, 82, 81, 0, 22, 14, 67, 63, 4, 26, 7, 62, 12, 80))
The ratio of between to total sum of square: 0.1905641377254551


#### 3.2.3 Max-p

The so-called max-p regions model (outlined in Duque, Anselin, and Rey 2012) uses a different approach and considers the regionalization problem as an application of integer programming. In addition, the number of regions is determined endogenously.

The algorithm itself consists of a search process that starts with an initial feasible solution and iteratively improves upon it while maintaining contiguity among the elements of each cluster. Like Geoda, pygeoda provides three different heuristic algorithms to find an optimal solution for max-p:

* greedy
* Tabu Search
* Simulated Annealing

Unlike SKATER and REDCAP that one can specify the number of clusters as an input paramter, max-p doesn't take this parameter but the constrained variable and the minimum value that each cluster should reach. 

For example, we use `greedy` algorithm in maxp function with the same dataset and weights as above to find optimal clusters using max-p. 

First, we need to specify, for example, every cluster must have population >= 3236.67 thousands people:

In [53]:
bound_vals = guerry.GetRealCol("Pop1831")
min_bound = 3236.67 # 10% of Pop1831
print(bound_vals)

(346.03, 513.0, 298.26, 155.9, 129.1, 340.73, 289.62, 253.12, 246.36, 270.13, 359.06, 359.47, 494.7, 258.59, 362.53, 445.25, 256.06, 294.83, 375.88, 598.87, 265.38, 482.75, 265.54, 299.56, 424.25, 278.82, 524.4, 357.38, 427.86, 312.16, 554.23, 346.3, 547.05, 245.29, 297.02, 550.26, 312.5, 281.5, 235.75, 391.22, 292.08, 470.09, 305.28, 283.83, 346.89, 140.35, 467.87, 591.28, 337.08, 249.83, 352.59, 415.57, 314.59, 433.52, 417.0, 282.52, 989.94, 397.73, 441.88, 655.22, 573.11, 428.4, 233.03, 157.05, 540.21, 424.26, 434.43, 338.91, 523.97, 457.37, 935.11, 693.68, 323.89, 448.18, 297.85, 543.7, 333.84, 242.51, 317.5, 239.11, 330.36, 282.73, 285.13, 397.99, 352.49)


Then, we can call the max-p function with "greedy" algorithm,
the bound values and minimum bound value:

In [54]:
maxp_clusters = pygeoda.maxp(queen_w, data, bound_vals, min_bound, "greedy")

betweenss = pygeoda.between_sumofsquare(maxp_clusters, data)
ratio = betweenss / totalss
print("The ratio of between to total sum of square:", ratio)

The ratio of between to total sum of square: 0.507018079733202


We can also specify using `tabu search` algorithm in maxp function
with the parameter of tabu length:

In [55]:
maxp_tabu_clusters = pygeoda.maxp(queen_w, data, bound_vals, min_bound, "tabu", tabu_length=95)

betweenss = pygeoda.between_sumofsquare(maxp_tabu_clusters, data)
ratio = betweenss / totalss
print("The ratio of between to total sum of square:", ratio)

The ratio of between to total sum of square: 0.5280299052291919


To apply `simulated annealing` algorithm in maxp function with
the parameter of cooling rate:

In [56]:
maxp_sa_clusters = pygeoda.maxp(queen_w, data, bound_vals, min_bound, "sa", cool_rate=0.75)

betweenss = pygeoda.between_sumofsquare(maxp_sa_clusters, data)
ratio = betweenss / totalss
print("The ratio of between to total sum of square:", ratio)

The ratio of between to total sum of square: 0.5260248902417842


We can also increase the number of iterations for local search process by specifying the parameter `initial` (default value is 99):


In [57]:
maxp_clusters = pygeoda.maxp(queen_w, data, bound_vals, min_bound, "greedy", initial=1000)

betweenss = pygeoda.between_sumofsquare(maxp_clusters, data)
ratio = betweenss / totalss
print("The ratio of between to total sum of square:", ratio)

The ratio of between to total sum of square: 0.5260248902417843
